<a href="https://colab.research.google.com/github/zahranorozzadeh/DeepLearning44/blob/main/Persian_Recognition_ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip3 install wandb

In [ ]:
import cv2
import os
import wandb
import numpy as np
import cv2
import tensorflow as tf
from wandb.keras import WandbCallback
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
from tensorflow.keras.layers import Dense, MaxPool2D, Conv2D, Flatten
from tensorflow.keras.optimizers import SGD, Adam

In [ ]:
folder_address = '/content/drive/MyDrive/Persian_Recognition/'


In [ ]:
wandb.init(project='Persian_Recognition')

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
config = wandb.config
config.learning_rate = 0.001

In [ ]:
width = 224
height = 224

In [ ]:
batch_size = 64

In [ ]:
data_generator = ImageDataGenerator(
    rescale = 1 / 255,
    validation_split = 0.2,
    horizontal_flip = True
)
train_data = data_generator.flow_from_directory(folder_address,
                                                    target_size = (width, height),
                                                    class_mode = 'categorical',
                                                    batch_size = batch_size,
                                                    shuffle = True,
                                                    subset ='training')

val_data = data_generator.flow_from_directory(folder_address,
                                                    target_size = (width, height),
                                                    class_mode = 'categorical',
                                                    batch_size = batch_size,
                                                    shuffle = True,
                                                    subset ='validation')

Found 2338 images belonging to 2 classes.
Found 584 images belonging to 2 classes.


In [ ]:
base_model = tf.keras.applications.ResNet50V2(
    input_shape=(width, height, 3),
    include_top = False,
    weights = 'imagenet',
    pooling = 'max'
    )

94683136/94668760 [==============================] - 2s 0us/step


In [ ]:

base_model.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
_________________________________________________________________________________________

In [ ]:

for layer in base_model.layers[:-8]:
  layer.trainable = False

In [ ]:

model = tf.keras.models.Sequential([
    base_model,
    Dense(2, activation='softmax')                          
])

In [ ]:
model.compile(optimizer = tf.keras.optimizers.SGD(),
              loss = tf.keras.losses.binary_crossentropy,
              metrics='accuracy')

In [ ]:

model.fit(train_data,
          steps_per_epoch = train_data.samples / batch_size,
          validation_data = val_data,
          validation_steps = val_data.samples / batch_size,
          epochs = 12,
          class_weight = {0:1, 1:3},
          callbacks = [WandbCallback()])

Epoch 1/12
36/36 [==============================] - 1117s 30s/step - loss: 2.6236 - accuracy: 0.8666 - val_loss: 2.4997 - val_accuracy: 0.6695


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/12
36/36 [==============================] - 29s 785ms/step - loss: 0.2150 - accuracy: 0.9598 - val_loss: 0.7696 - val_accuracy: 0.8373
Epoch 3/12
36/36 [==============================] - 29s 801ms/step - loss: 0.1110 - accuracy: 0.9778 - val_loss: 0.4016 - val_accuracy: 0.8990
Epoch 4/12
36/36 [==============================] - 29s 802ms/step - loss: 0.0643 - accuracy: 0.9867 - val_loss: 0.4262 - val_accuracy: 0.8836
Epoch 5/12
36/36 [==============================] - 29s 787ms/step - loss: 0.0598 - accuracy: 0.9880 - val_loss: 0.3518 - val_accuracy: 0.8955
Epoch 6/12
36/36 [==============================] - 29s 780ms/step - loss: 0.0390 - accuracy: 0.9940 - val_loss: 0.3458 - val_accuracy: 0.9041
Epoch 7/12
36/36 [==============================] - 28s 773ms/step - loss: 0.0243 - accuracy: 0.9966 - val_loss: 0.3401 - val_accuracy: 0.8973
Epoch 8/12
36/36 [==============================] - 28s 770ms/step - loss: 0.0196 - accuracy: 0.9979 - val_loss: 0.3432 - val_accuracy: 0.9024

In [ ]:
model.save('/content/model.h5')